In [1]:
import pygame_2d
import numpy as np
from const import*

screen = np.zeros((720, 1280, 3), dtype = np.uint8)



pygame 2.5.2 (SDL 2.28.3, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

high, low = 1,0

alpha = 0, 2*math.pi    # góc quay 
rVelo = PLAYER_SETTING.MIN_ROTATION_VELO, PLAYER_SETTING.MAX_ROTATION_VELO  # vận tốc quay 
fwVelo = -PLAYER_SETTING.MAX_FORWARD_VELO, PLAYER_SETTING.MAX_FORWARD_VELO    # vận tốc tiến



In [3]:

gamma = 0.99
alpha = 0.1
epsilon = 1
epsilon_decay = epsilon / 4000
n_epsilondes = 10000



In [9]:
random.seed(13)
lidars = np.random.randint(200, 500, size= 360)
print(lidars)

bin_lidarsignal = np.linspace(
    0, 360, num=LENGTH_LIDARSIGNAL, endpoint=True)
bin_lidarsignal = np.delete(bin_lidarsignal, 0)
lidars_digitized = np.digitize(lidars, bin_lidarsignal)
print(lidars_digitized)

bin_lidarspace = np.array([45, 135, 180])
lidars_sections = np.array_split(lidars_digitized, bin_lidarspace)
section_lidars_min = [np.amin(section) for section in lidars_sections]
print(section_lidars_min)



[308 466 325 446 471 342 359 281 313 400 448 313 482 353 492 403 371 311
 355 474 327 381 478 257 393 322 491 467 401 350 351 264 483 217 242 409
 319 281 247 226 306 468 305 488 411 385 252 295 277 231 449 318 314 453
 263 292 398 393 297 457 467 417 463 216 487 499 283 258 256 377 361 353
 397 428 200 403 378 318 454 390 303 409 488 413 380 264 308 352 421 330
 459 480 496 441 225 428 416 271 318 379 229 447 380 455 398 202 410 390
 421 455 371 314 275 436 273 307 341 447 357 245 240 465 205 451 496 480
 246 232 350 373 438 280 327 429 421 457 328 438 293 283 268 209 389 302
 239 278 468 244 433 258 277 361 325 303 439 437 494 261 453 433 429 394
 457 393 384 432 351 301 311 460 233 245 445 409 388 430 486 294 474 413
 243 339 438 256 447 316 350 210 261 440 200 232 326 282 269 400 332 489
 430 415 340 331 217 356 218 498 397 265 464 212 237 233 478 242 484 393
 386 332 387 354 300 207 234 376 481 310 268 274 427 249 202 286 490 275
 285 234 271 382 245 238 241 375 258 274 252 292 38

In [ ]:
bin_lidarspace = np.array([45, 135, 180])
lidars_sections = np.array_split(lidars_digitized, bin_lidarspace)
section_lidars_min = [np.amin(section) for section in lidars_sections]


array([    0,     0,     0, 99999], dtype=int64)

In [ ]:

high, low = 1, 0
alpha_space = 0, 2*math.pi
fwVelo_space = -PLAYER_SETTING.MAX_FORWARD_VELO, PLAYER_SETTING.MAX_FORWARD_VELO
rVelo_space = PLAYER_SETTING.MIN_ROTATION_VELO, PLAYER_SETTING.MAX_ROTATION_VELO

lowState = np.array(
    [alpha_space[0], fwVelo_space[0], rVelo_space[0]], dtype=float)
upState = np.array(
    [alpha_space[1], fwVelo_space[1], rVelo_space[1]], dtype=float)

infoState_shape = (ALPHA_SPACE, FWVELO_SPACE, RVELO_SPACE)

bins = []
for i in range(3):
    item = np.linspace(
        lowState[i],
        upState[i],
        num=infoState_shape[i],
        endpoint=False)
    item = np.delete(item, 0)
    bins.append(item)
    print(bins[i])


[0.31415927 0.62831853 0.9424778  1.25663706 1.57079633 1.88495559
 2.19911486 2.51327412 2.82743339 3.14159265 3.45575192 3.76991118
 4.08407045 4.39822972 4.71238898 5.02654825 5.34070751 5.65486678
 5.96902604]
[-50.   0.  50.]
[-0.5  0.   0.5]


In [ ]:
alpha = 0.3993
fwVelo = 23
rVelo = 0.32
s = (alpha, fwVelo, rVelo)
infoStateVector = []
for i in range(3):
    infoStateVector.append(np.digitize(s[i], bins[i]))

infoStateVector = np.array(infoStateVector)
infoStateVector

array([1, 2, 2], dtype=int64)

In [ ]:
np.concatenate((infoStateVector, lidarStateVector))

array([    1,     3,     0,     0,     0,     0, 99999], dtype=int64)

In [ ]:
new_observation_shape = (ALPHA_SPACE, FWVELO_SPACE, RVELO_SPACE, SECTIONS_LIDARSPACE)
q_table = np.zeros(new_observation_shape + (ACTION_SPACE,))
q_table.shape

(20, 4, 4, 4, 6)